**思路**

1、修正方向。

1、剔除无效的ship_ID:

    train剔除，test不管。
    
    以三种作业方式各自的坐标阈值为条件剔除。
    
    拖网：
    
    围网：
    
    刺网：
    
 2、判断作业时段和非作业时段：
 
    待思考？
    
 3、GPS异常值剔除/修正：

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import os
from tqdm import tqdm

import warnings

import matplotlib.pyplot as plt

from datetime import datetime as dt

warnings.filterwarnings('ignore')

In [2]:
train = pd.read_hdf('C:/Users/f3107/Desktop/hy_data/train_descending_deltatime.h5')
train['datetime'] = pd.to_datetime(train['datetime'],train['delta_time'], format='%m%d %H:%M:%S')
train['delta_time'] = train['delta_time'].dt.total_seconds().astype('int')

test = pd.read_hdf('C:/Users/f3107/Desktop/hy_data/test_descending_deltatime.h5')
test['datetime'] = pd.to_datetime(test['datetime'],test['delta_time'], format='%m%d %H:%M:%S')
test['delta_time'] = test['delta_time'].dt.total_seconds().astype('int')

#### 方向修正

In [3]:
def delta_direction(df):
    
    df['delta_direction'] = abs(df['direction'].diff())
    df['delta_direction'] = 180-abs(df['delta_direction']-180)
    df['delta_direction'].fillna(0,inplace=True)
    
    return df

In [4]:
train = delta_direction(train)
test = delta_direction(test)

#### 剔除无效的ship_ID

In [5]:
def del_ship(train):
    
    train_label = train.drop_duplicates('ship_ID')

    train_x = train['x'].groupby(train['ship_ID']).agg(['max','min']).reset_index().rename(columns = {'max':'x_max','min':'x_min'})
    train_y = train['y'].groupby(train['ship_ID']).agg(['max','min']).reset_index().rename(columns = {'max':'y_max','min':'y_min'})
    train_x['x_max_x_min'] = train_x['x_max'] - train_x['x_min']
    train_y['y_max_y_min'] = train_y['y_max'] - train_y['y_min']

    train_data = pd.merge(train_label[['ship_ID','type']], train_x, on ='ship_ID')
    train_data = pd.merge(train_data, train_y, on ='ship_ID')

    train_data_tuo = train_data[train_data['type']=='拖网']
    del_tuo = list(train_data_tuo['ship_ID'][(train_data_tuo['x_max_x_min']<500)&(train_data_tuo['y_max_y_min']<500)])

    train_data_ci = train_data[train_data['type']=='刺网']
    del_ci = list(train_data_ci['ship_ID'][(train_data_ci['x_max_x_min']<100)&(train_data_ci['y_max_y_min']<100)])

    train_data_wei = train_data[train_data['type']=='围网']
    del_wei = list(train_data_wei['ship_ID'][(train_data_wei['x_max_x_min']<100)&(train_data_wei['y_max_y_min']<100)])

    del_ship_ID = del_tuo+del_ci+del_wei
    
    return del_ship_ID

In [6]:
del_ship_ID = del_ship(train)
train = train[~train.ship_ID.isin(del_ship_ID)]